In [33]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer

In [67]:
df=pd.read_csv(r"https://raw.githubusercontent.com/lukasgarbas/nlp-text-emotion/master/data/data_train.csv")

In [68]:
df=df.sample(len(df))

In [69]:
multilabel=MultiLabelBinarizer()
y=multilabel.fit_transform(df['Emotion'])

In [70]:
one_hot_encoded_data = pd.get_dummies(df, columns = ['Emotion'])

In [71]:
y=one_hot_encoded_data
y=y.drop(columns=["Text","Emotion_anger"])
y

,Emotion_fear,Emotion_joy,Emotion_neutral,Emotion_sadness
2550,False,False,False,False
3692,False,False,False,True
2602,False,False,False,False
5934,False,False,True,False
2295,False,False,False,False
...,...,...,...,...
6735,False,False,False,True
4696,False,False,True,False
5846,False,False,False,True
2615,False,True,False,False


In [72]:
tfidf=TfidfVectorizer(analyzer='word',max_features=5000,ngram_range=(1,3),stop_words='english')
x=tfidf.fit_transform(df['Text'])

In [73]:
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.2,random_state=3)

In [74]:
xtrain.shape,xtest.shape,ytrain.shape,ytest.shape

((6347, 5000), (1587, 5000), (6347, 4), (1587, 4))

In [87]:
lr=LogisticRegression(solver='newton-cg')


In [88]:
from sklearn.multiclass import OneVsRestClassifier

In [89]:
clf=OneVsRestClassifier(lr)
clf.fit(xtrain,ytrain)

OneVsRestClassifier(estimator=LogisticRegression(solver='newton-cg'))

In [96]:
ypred = clf.predict(xtest)

In [91]:
x=['please forgive me']
xt=tfidf.transform(x)
clf.predict(xt)

array([[0, 0, 0, 0]])

In [92]:
from sklearn.metrics import accuracy_score
print(accuracy_score(ytest,clf.predict(xtest)))

0.424700693131695


In [107]:
from sklearn.ensemble import ExtraTreesClassifier

model = ExtraTreesClassifier(verbose=0, random_state=42, n_jobs=-1)
model.fit(xtrain, ytrain)

ExtraTreesClassifier(n_jobs=-1, random_state=42)

In [108]:
model.score(xtest, ytest)

0.6212980466288595

In [109]:
parameters = {
    'n_estimators': [10, 20, 50, 100, 200, 500, 1000, 1200, 1500, 1800, 1900, 2000, 2100, 3000],
    'criterion': ['gini', 'entropy'],
    'max_depth': [1, 2, 5, 8, 13, 21, 34, 53, 54, 55, 89, None],
    'min_samples_split': [2, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377],
    'min_samples_leaf': [1, 2, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377],
    'min_weight_fraction_leaf': [x / 10 for x in range(0, 6)],
    'max_features': ['auto', 'sqrt', 'log2', 2, 5, 8, 13, 21, 34, None],
    'max_leaf_nodes': [2, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, None],
    'min_impurity_decrease': [x / 100 for x in range(0, 11)],
    'bootstrap': [True, False],
    'oob_score': [True, False],
    'warm_start': [True, False],
    'class_weight': ['balanced', 'balanced_subsample', None]
}

In [113]:
from sklearn.model_selection import RandomizedSearchCV

clf = RandomizedSearchCV(model, parameters, cv=5, verbose=5, n_jobs=-1)
clf.fit(xtrain, ytrain)

OverflowError: Python int too large to convert to C long

In [ ]:
clf.score(xtest, ytest)